# Data Cleaning

### Imports

In [1]:
import pandas as pd

### Load Data

In [2]:
posts = pd.read_csv('data/all_sports_posts.csv')

In [3]:
posts

,title,subreddit,score,author_flair_text,link_flair_text,created_utc,num_comments,age
0,Rivalry renewed: Canada advances to face U.S. ...,sports,1,NaN,:hockey: Hockey,1.662233e+09,0,750.568575
1,Multiple Games on any TV with Chrome - Quick t...,sports,0,NaN,:football: Football,1.662232e+09,1,1556.568575
2,Google Deepmind AI Learns Soccer To Power Robo...,sports,1,NaN,:soccer: Soccer,1.662229e+09,0,4977.568575
3,"FIFA, Qatar agree to beer sales at World Cup",sports,131,NaN,:soccer: Soccer,1.662222e+09,23,11504.568575
4,Old Dominion does it again in second upset of ...,sports,298,:BIG12isu: Iowa State,:football: Football,1.662209e+09,10,25210.568575
...,...,...,...,...,...,...,...,...
19771,First set of clubs,golf,2,NaN,EQUIPMENT,1.661813e+09,6,422496.561974
19772,"Bought a used set of stealths off 2nd swing, o...",golf,77,NaN,EQUIPMENT,1.661812e+09,20,423590.561974
19773,New Taylormade irons release?,golf,0,NaN,EQUIPMENT,1.661812e+09,1,423744.561974
19774,How do you breath during your swing?,golf,4,NaN,NaN,1.661812e+09,20,423974.561974


### Inspect Data Types and Missing Values

In [4]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19776 entries, 0 to 19775
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              19776 non-null  object 
 1   subreddit          19776 non-null  object 
 2   score              19776 non-null  int64  
 3   author_flair_text  8393 non-null   object 
 4   link_flair_text    10326 non-null  object 
 5   created_utc        19776 non-null  float64
 6   num_comments       19776 non-null  int64  
 7   age                19776 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 1.2+ MB


In [5]:
posts.subreddit.value_counts()

MMA                  999
tennis               998
snowboarding         996
soccer               995
CollegeBasketball    995
Fishing              993
running              993
hockey               993
CFB                  992
NASCAR               991
golf                 991
nfl                  990
bicycling            989
formula1             989
Boxing               986
baseball             986
nba                  985
skiing               985
climbing             980
sports               950
Name: subreddit, dtype: int64

Cleaning:
- DONE Eliminate duplicate posts (posts with same title) using pd.drop_duplicates
    - keep last to eliminate re-posts
- DONE Create columns for whether post has:
    - author flair text
    - link flair text
- DONE Lowercase all subreddit values
    - MMA
    - CollegeBasketball
    - Fishing
    - CFB
    - NASCAR
    - Boxing
- DONE Drop original author, link flair text columns

In [6]:
posts.duplicated(subset=['title'], keep=False).value_counts()

False    17468
True      2308
dtype: int64

There are 2308 posts of which 2 or more copies exist.

In [7]:
posts.duplicated(subset=['title'], keep='last').value_counts()

False    18478
True      1298
dtype: int64

There are 1298 non-lasts, which we want to drop, and therefore 1010 lasts (out of 2308) that we want to keep.

In [8]:
posts.drop_duplicates(subset=['title'], keep='last', ignore_index=True).subreddit.value_counts()

hockey               993
soccer               990
Fishing              990
golf                 989
nfl                  986
formula1             985
CFB                  983
nba                  975
tennis               973
bicycling            970
NASCAR               966
baseball             960
skiing               959
snowboarding         954
Boxing               935
MMA                  923
running              836
CollegeBasketball    835
climbing             832
sports               444
Name: subreddit, dtype: int64

The cause of duplication is unclear, but assuming that it's either that subreddits contain re-posts or that the post collection algorithm collected the same post more than once, keeping lasts is best. We want either the oldest version of a re-posted post (more time up will translate to a more accurate picture of how its characteristics affect engagement with it), or else the versions are the same and it doesn't matter which we keep.

It is also unclear why the sports subreddit had by far the most duplicates, but given that it has the most subscribers, a higher prevalence of re-posting would not be surprising. It's worth noting that the other unique thing about this subreddit is that its data was collected first, but I can think of no reason that this would produce more duplicates. Finally, because we're losing a disproportionate number of posts from the subreddit that is not specific to any sport, this is unlikely to imbalance the data set in a way that would bias analysis.

In [9]:
# eliminate duplicate posts
posts = posts.drop_duplicates(subset=['title'], keep='last', ignore_index=True)

In [10]:
# create columns for whether the post has author, link flair text
posts['has_authtext'] = [1 - int(type(text) == float) for text in posts.author_flair_text]
posts['has_linktext'] = [1 - int(type(text) == float) for text in posts.link_flair_text]

C:\Users\gtayl\AppData\Local\Temp\ipykernel_228\3712620235.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts['has_authtext'] = [1 - int(type(text) == float) for text in posts.author_flair_text]
C:\Users\gtayl\AppData\Local\Temp\ipykernel_228\3712620235.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts['has_linktext'] = [1 - int(type(text) == float) for text in posts.link_flair_text]


In [11]:
# lowercase all subreddit values
posts['subreddit'] = [sub.lower() for sub in posts.subreddit]

C:\Users\gtayl\AppData\Local\Temp\ipykernel_228\666270466.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts['subreddit'] = [sub.lower() for sub in posts.subreddit]


In [12]:
# drop original author, link flair text columns
posts = posts.drop(columns=['author_flair_text', 'link_flair_text'])

In [13]:
posts

,title,subreddit,score,created_utc,num_comments,age,has_authtext,has_linktext
0,Serena Williams’s Once-In-A-Lifetime Serve,sports,1,1.661814e+09,0,420116.161326,1,1
1,"Wholesome moment, ""Ichiro Girl"" throws out fir...",sports,1413,1.661797e+09,50,436886.161326,0,1
2,Williams' goodbye &amp; Raducanu's defence top...,sports,5,1.661797e+09,1,437390.161326,0,1
3,Duke volleyball player: BYU slow to act on slurs,sports,4962,1.661787e+09,703,447296.161326,1,1
4,[ESPN] Washington Commanders RB Brian Robinson...,sports,55,1.661743e+09,5,490948.161326,1,1
...,...,...,...,...,...,...,...,...
18473,First set of clubs,golf,2,1.661813e+09,6,422496.561974,0,1
18474,"Bought a used set of stealths off 2nd swing, o...",golf,77,1.661812e+09,20,423590.561974,0,1
18475,New Taylormade irons release?,golf,0,1.661812e+09,1,423744.561974,0,1
18476,How do you breath during your swing?,golf,4,1.661812e+09,20,423974.561974,0,0


In [14]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18478 entries, 0 to 18477
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         18478 non-null  object 
 1   subreddit     18478 non-null  object 
 2   score         18478 non-null  int64  
 3   created_utc   18478 non-null  float64
 4   num_comments  18478 non-null  int64  
 5   age           18478 non-null  float64
 6   has_authtext  18478 non-null  int64  
 7   has_linktext  18478 non-null  int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 1.1+ MB


No missing values, data types correct, all columns potentially useful. Data clean!

In [16]:
posts.to_csv('data/clean_sports_posts.csv', index=False)